In [1]:
import os
import time
import inspect

import tensorflow as tf
import numpy as np
from scipy.io import loadmat

import skimage
import skimage.io
import skimage.transform

import warnings
warnings.filterwarnings('ignore')

import scipy
import matplotlib.pyplot as plt
plt.ion()

In [66]:
class VggFace2:
    def __init__(self, vggFace2_mat_path=None):
        self.mean_pixel = np.array([131.0912,  103.8827,   91.4953])
        
#         if vggFace2_mat_path is None:
#             path = inspect.getfile(VggFace2)
#             path = os.path.abspath(os.path.join(path, os.pardir))
#             path = os.path.join(path, "./matconvnet/senet/senet50_ft-dag.mat")
#             vggFace2_mat_path = path
#             print(vggFace2_mat_path)

#         print("mat file loading...")
        data = loadmat("./matconvnet/senet/senet50_ft-dag.mat")
        
#         print("make data_dict...")
        self.data_dict = {}
        for idx in range(len(data['params'][0])):
            param_name = data['params'][0][idx][0][0]
            param_value = data['params'][0][idx][1]
            self.data_dict[param_name] = param_value
        
#         print("make layer info...")
        self.layer_info = {}
        self.layers = data['layers'][0]
        for idx in range(len(data['layers'][0])):
            per_layer_info = {}
            
            layer_name = self.layers[idx]['name'][0]
            layer_type = self.layers[idx]['type'][0]
            input_name = self.layers[idx][2][0][0][0]
            output_name = self.layers[idx][3][0][0][0]            
            params_names = []
            if len(self.layers[idx][4]) > 0:
                for i in range(len(self.layers[idx][4][0])):
                    params_names.append(self.layers[idx][4][0][i][0])
            meta_info = {}
            if self.layers[idx][5] == None:
                meta_info = {None}
            else:
                for meta_info_name in self.layers[idx][5].dtype.names:
                    if meta_info_name in ['numChannels', 'opts']:
                        meta_info[meta_info_name] = self.layers[idx][5][meta_info_name][0][0]
                    elif meta_info_name in ['pad', 'dilate', 'size', 'stride', 'method', 'poolSize', 'shape']:
                        meta_info[meta_info_name] = self.layers[idx][5][meta_info_name][0][0][0]
                    elif meta_info_name in ['epsilon', 'hasBias', 'useShortCircuit', 'leak']:
                        meta_info[meta_info_name] = self.layers[idx][5][meta_info_name][0][0][0][0]
                    else:
                        meta_info[meta_info_name] = self.layers[idx][5][meta_info_name][0][0][0][0][0]
            per_layer_info['name'] = layer_name
            per_layer_info['type'] = layer_type
            per_layer_info['input_name'] = input_name
            per_layer_info['output_name'] = output_name
            per_layer_info['params'] = params_names
            per_layer_info['meta'] = meta_info
            self.layer_info[layer_name] = per_layer_info

#     def _undo():
#         out_for_print = out + np.array([131.0912,  103.8827,   91.4953])
#         out_for_print = out_for_print/255.0

    def build(self, input_image, isImageNormalized=True):
        """
        :param input_image: rgb image [batch, height, width, 3] values scaled [0,1] - when use skimage
        """
        start_time = time.time()
        print("build model started...")
        if isImageNormalized:
            input_image = input_image * 255.0
        
        input_image = input_image - self.mean_pixel
        
        if not (input_image.get_shape().as_list()[1:] == [224, 224, 3]):
            input_image = tf.image.resize_bicubic(input_image, [224, 224])

        current = input_image
        for idx, layer in enumerate(self.layers):
            name = self.layers['name'][idx][0]
            layer_type = self.layers['type'][idx][0]
            if layer_type == 'dagnn.Conv':
                current = self.conv_layer(current, name)
                break
#                 print(self.layer_info[name]['meta']['pad'])
#                 print(self.layer_info[name]['meta']['size'])
#             if name[:4] == 'conv':
#                 print(self.layer_info[name]['meta'])
        return current

    def conv_layer(self, bottom, name):
        with tf.variable_scope(name):
            kernel = self.get_conv_kernel(name)
            bias = self.get_conv_bias(name)
            stride = self.get_conv_stride(name)
            
            conv = tf.nn.conv2d(bottom, kernel, stride, padding='SAME')
            return tf.nn.bias_add(conv, bias)

    def get_conv_kernel(self, name):
        # matconvnet: kernels are [width, height, in_channels, out_channels]
        # tensorflow: kernels are [height, width, in_channels, out_channels]
        kernel = self.data_dict[name + '_filter']
        kernel = np.transpose(kernel, (1, 0, 2, 3))
        
        return tf.constant(kernel, name="kernels")

    def get_conv_stride(self, name):
        s = self.layer_info[name]['meta']['stride'][0]
        print("stride: ", [1, s, s, 1])
        return [1, s, s, 1]
        
    def get_conv_bias(self, name):
        if self.layer_info[name]['meta']['hasBias']:
            return tf.constant(self.data_dict[name + '_bias'], name="bias")
        else:
            bias_size = self.layer_info[name]['meta']['size'][3]
            return tf.zeros([bias_size], tf.float32, name="bias")

        #all padding is "SAME"
#     def get_conv_pad(self, name):
#         return self.layer_info[name]['meta']['pad']

In [8]:
def load_image(path):
    # load image
    img = skimage.io.imread(path)
    # print "Original Image Shape: ", img.shape
    # we crop image from center
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    # resize to 224, 224
    resized_img = skimage.transform.resize(crop_img, (128, 128))
    return resized_img

In [4]:
img1 = load_image("./data/61237.jpg")
img1 = img1.reshape((1, 128, 128, 3))

img2 = load_image("./data/62607.jpg")
img2 = img2.reshape((1, 128, 128, 3))

batch = np.concatenate((img1, img2), 0)

In [67]:
with tf.Session() as sess:
    image = tf.placeholder("float", [2, 128, 128, 3])
    feed_dict = {image: batch}
    
    vggFace2 = VggFace2()
    vggFace2_out = vggFace2.build(image)
    out = sess.run(vggFace2_out, feed_dict=feed_dict)

build model started...
64
stride:  [1, 2, 2, 1]
64


In [64]:
out.shape

(2, 112, 112, 64)

In [65]:
out

array([[[[  4.31169662e+01,   1.42861242e+01,  -5.52621117e+01, ...,
            1.75367851e+01,   2.04803772e+01,   1.14209194e+01],
         [  3.59034996e+01,   1.75721550e+01,  -6.18541031e+01, ...,
            2.00557556e+01,   3.57287955e+00,   3.49040568e-01],
         [  3.58153763e+01,   1.73158607e+01,  -6.09353981e+01, ...,
            1.95153217e+01,   3.06573343e+00,  -8.49547267e-01],
         ..., 
         [  4.08334923e+01,   1.88881588e+01,  -7.05099945e+01, ...,
            2.23978500e+01,   5.86627531e+00,   3.26881409e-02],
         [  3.16866226e+01,   1.06459188e+01,  -5.64001770e+01, ...,
            1.73070202e+01,  -7.71646118e+00,  -2.68686676e+01],
         [  9.61083984e+00,   6.34740412e-01,  -3.15489159e+01, ...,
            1.30664749e+01,  -1.86040001e+01,   6.08847313e+01]],

        [[  5.18645554e+01,   2.03697453e+01,   3.31753230e+00, ...,
            1.19935522e+01,   2.51655083e+01,   6.64165401e+00],
         [  4.35081139e+01,   2.50342808e+01,

In [ ]:
out_for_print = out + np.array([131.0912,  103.8827,   91.4953])
out_for_print = out_for_print/255.0

plt.imshow(scipy.misc.toimage(np.squeeze(img1)))